In [2]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: nltk in c:\users\sandr\anaconda3\lib\site-packages (3.7)



In [3]:
pip install praw

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import re 
import string 
from string import punctuation 
import nltk 
from nltk.corpus import stopwords
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer 
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation, Dropout 
from tensorflow.keras.callbacks import EarlyStopping
from nltk.sentiment import SentimentIntensityAnalyzer
import praw
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [6]:

# Read the CSV file
df = pd.read_csv('ProjectTweets.csv')

# Display the first few rows of the DataFrame
df.head(10)


,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,5,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
5,6,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
6,7,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
7,8,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
8,9,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?
9,10,1467812416,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing


In [7]:
print(df.columns)


Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')


In [8]:

#  Ignore the headers of the DataFrame
df = pd.read_csv('ProjectTweets.csv', header=None)

#  My column names
df_columns = ['Id', 'Code', 'Date', 'No_Query','User','Comments']  

# column names to the DataFrame
df.columns = df_columns

df.head()


,Id,Code,Date,No_Query,User,Comments
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
df = df.drop(['Id', 'No_Query', ], axis=1)

df.dropna(inplace=True)

df.head()

,Code,Date,User,Comments
0,1467810369,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
2,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
3,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
4,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."


# Data Pre Processing

In [10]:
from nltk.corpus import stopwords
nltk.download('stopwords')

import string
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sandr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# Store the stopwords into the object named as "stop_words"
stop_words = stopwords.words('english')

# Store the string.punctuation into an object punct
punct = string.punctuation

# Initialise an object using a method PorterStemmer
stemmer = PorterStemmer()

In [13]:

# Initialize stopwords, stemmer, and punctuation
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
punct = string.punctuation

cleaned_data = []

for text in text_data:
    # Remove non-alphabetic characters and convert to lowercase
    text = re.sub('[^a-zA-Z]', ' ', text).lower()
    
    # Tokenize the text into words
    words = text.split()
    
    # Apply stemming, remove stopwords, and punctuation
    cleaned_words = [stemmer.stem(word) for word in words if (word not in stop_words) and (word not in punct)]
    
    # Join the cleaned words back into a string
    cleaned_text = ' '.join(cleaned_words)
    
    cleaned_data.append(cleaned_text)



NameError: name 'text_data' is not defined

In [ ]:

# Define a function to preprocess text
def preprocess_text(text):
    # Remove special characters and lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    return text

# preprocessing to the 'Comments' column
df['Comments'] = df['Comments'].apply(preprocess_text)

# Define a function to tokenize words
def tokenize_words(text):
    return word_tokenize(text)

# Apply word tokenization to the 'Comments' column
df['Comments'] = df['Comments'].apply(tokenize_words)
df.head()


In [ ]:
# Define a stemming function
def normalize_words(text):
    stemmer = nltk.PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in text]
    return stemmed_words

# Apply word stemming to the 'Comments' column
df['Comments'] = df['Comments'].apply(normalize_words)


In [ ]:
# Define a function to remove stop words
def remove_stopwords(words):
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    return filtered_words

# Apply stop word removal to the 'Comments' column
df['Comments'] = df['Comments'].apply(remove_stopwords)


In [ ]:
# Join the tokenized and preprocessed words into a single string
df['Comments'] = df['Comments'].apply(lambda words: ' '.join(words))

# Create a CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the 'Comments' column to create a document-term matrix
X = vectorizer.fit_transform(df['Comments'])
